# Step 3
### Saving LinkedIn's Profile Data into Text Files

In [ ]:
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def get_linkedin_data(extracted_linkedin_url, metadata):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Enables headless mode

    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get(extracted_linkedin_url)
        html_content = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html_content, 'html.parser')

    title = soup.title.string if soup.title else "Title not found"

    # Find all <script type="application/ld+json"> tags and extract the JSON from the first one
    script_tags = soup.find_all('script', type="application/ld+json")
    json_str = script_tags[0].text  # Use .text or .string to get the content of the tag
    data = json.loads(json_str)  # Parse the JSON string
    
    # Initialize an empty string to accumulate the extracted data
    extracted_data = ""

    extracted_data += f"Title: {title}\n\n"

    # Extract the meta description content
    meta_description = soup.find('meta', {'name': 'description'})['content'] if soup.find('meta', {'name': 'description'}) else "Description not found"
    extracted_data += f"- Description: {meta_description}\n"

    # Assuming the person's info is the first entry in the "@graph" list
    person_info = data["@graph"][0]

    name = person_info.get("name", "")
    extracted_data += f"Name: {name}\n\n"

    # Extracting alumni and work information
    extracted_data += "Work Experience:\n"
    alumni_of = person_info.get("alumniOf", [])
    for alumni in alumni_of:
        org_name = alumni.get("name", "")
        member = alumni.get("member", {})
        role = member.get("description", "")  # Access description
        extracted_data += f"- Organization: {org_name}, Role: {role}\n\n"

    member_of = person_info.get("memberOf", [])
    for member in member_of:
        org_name = member.get("name", "")
        role = member.get("description", "")
        extracted_data += f"- Organization: {org_name}, Role: {role}\n\n"

    # Extracting languages
    extracted_data += "\nLanguages Known:\n"
    languages = person_info.get("knowsLanguage", [])
    for language in languages:
        language_name = language.get("name", "")
        extracted_data += f"- {language_name}\n"
    
    file_name = f"{metadata['first_name']}_{metadata['last_name']}.txt"
    with open(file_name, "w") as file:
        file.write(extracted_data)
        
    return extracted_data